# Image processing pipeline

In [ ]:
%matplotlib inline
from matplotlib import pyplot as plt

imgset_type = 'Y27_RhoA_ZO1'
imgset_filename = '2017.06.17_01'
imgset_local_path = 'Y27-ZO1/ZY6'

TSCALE = 5.03 # sec per frame
LSCALE = 0.248 # um per px

In [ ]:
from importlib import reload
from warnings import warn

import sys
import os
codebase_dir = './'
if codebase_dir not in sys.path:
    sys.path.append(codebase_dir)

import numpy as np
import cv2
import imageio as iio
import tiffcapture as tc
from scipy.signal import savgol_filter

from tqdm import tqdm

working_dir = '/path/to/image/files/' + imgset_local_path
working_dir = os.path.normpath(working_dir)

if not os.path.exists(working_dir):
    warn("Directory {} doesn't exist".format(working_dir))

imgset_shortname = os.path.basename(working_dir)

def imgname(name='', pfx='b', ext='tif', mid='C'):
    postfix = "{mid}{pfx}{name}.{ext}".format(
        mid=mid, name=name, pfx=pfx, ext=ext)
    return os.path.join(working_dir, imgset_shortname + postfix)

main_imgfile = os.path.join(working_dir, imgset_shortname + '.tif')
flare_data_dir = os.path.join(working_dir, '..', '_flare-data')
flare_data_dir = os.path.normpath(flare_data_dir)
if not os.path.exists(flare_data_dir):
    os.makedirs(flare_data_dir)

## A. Cell geometry

In [ ]:
fullimage = iio.volread(main_imgfile)
jxn08U = np.uint16( fullimage[:,0] ) #+ fullimage[:,0] )
T, H, W = jxn08U.shape
# del fullimage

In [ ]:
# Tailing
jxnTail = np.zeros_like(jxn08U, dtype='uint16')

halflife = 1.5 #frames
hlcoeff = 0.5**(1.0/halflife)

prev = None
for i, dn in tqdm( enumerate( jxn08U ), total=T ):
    if prev is None:
          prev = np.zeros_like( dn, np.float32 )
    prev*=hlcoeff
    prev+=dn
    jxnTail[i] = prev
iio.volwrite( imgname('-1dn-tail'), jxnTail )

In [ ]:
# ThreshMarking algorithm fitting

import wflowtools.celldrawing as CD; reload(CD);
import wflowtools.cv2tools as cv2t; reload(cv2t);

def get_initial_labels( dn ):
    blur = cv2.blur(dn, (3,3) )
    thr = cv2t.otsu(blur, 0.6).astype('uint8')
    thr = cv2.morphologyEx(thr, cv2.MORPH_DILATE, kernel=np.ones( (3,3) ), iterations=2)
    labels, n_labels = cv2t.get_markers( 1 - thr)

    n_cells = cv2t.remove_small_components(labels, n_labels, area = 30)
    return labels

# dn = tiffio.image_from_tiff_stack( imgname('-1dn-tail'), i = 2 )
dn = jxnTail[0]
initial_labels = get_initial_labels( dn ).astype('uint16')
iio.imwrite(  imgname('-2lbl-0-pre-init'), initial_labels )
plt.imshow( initial_labels, cmap=CD.cells_cmap )
centers = CD.cell_centers( initial_labels )
centers.pop(0)
CD.draw_cells_labels( plt.gca(), initial_labels, centers=centers, color='white'  )

In [ ]:
initial_labels = np.clip( iio.imread(  imgname('-2lbl-0-pre-init')  ),  a_min = 0, a_max=1)
initial_labels, _ = cv2t.get_markers(initial_labels)
iio.imwrite(  imgname('-2lbl-0-init'), initial_labels )
centers = CD.cell_centers( initial_labels )
centers.pop(0)

dn = jxnTail[0]
wsh = cv2t.cwatershed(dn, initial_labels) 
smooth = cv2t.smooth_labels( wsh )
comp = CD.make_composite( smooth, wsh, dn, print_labels=True )
plt.imshow( comp )

In [ ]:
# Generate first labeled image

source = jxnTail
shape3d = source.shape

out = np.empty(shape3d+(3,), dtype='uint8' )
labels = np.empty( (shape3d[0]+1,) + shape3d[1:], dtype='uint16' )
labels[0] = labels[-1] = smooth

for t, dn in enumerate( tqdm( source ) ):
    wsh = cv2t.cwatershed(dn, labels[t-1] ) 
    labels[t] = cv2t.smooth_labels( wsh )
    out[t] = CD.make_composite( labels[t], wsh, dn, print_labels=True )

iio.volwrite( imgname('-2lbl-1'), out )

In [ ]:
# Correction step
import wflowtools.cv2tools as cv2t; reload(cv2t);

comp_in_fn  = '-2lbl-2'
comp_out_fn = '-2lbl-2'
T_from = 0
T_end = 600

comp_in  = iio.volread( imgname(comp_in_fn) )
jxnTail  = iio.volread( imgname('-1dn-tail'))
source = jxnTail
force_erode = {
    #     8: 2,
}

def labels_change(t, force_erode, labels=None):
    #     if t+1 == 138:
    #         force_erode[6] = 2
    #     if t+1 == 123:
    #         force_erode[26] = 2
    pass

gen = zip(range(T), source, comp_in)
# for t, img, comp in tqdm(gen, total=T):
#     print(comp)
# raise StopIteration()
n_cells = np.max(labels[0])
labels[-1] = smooth
prev_labels = 1*smooth

for t, img, comp in tqdm(gen, total=T):
    if t == T_from:
        n_cells = np.max(labels[t-1])
    if t >= T_from and t<=T_end:
        mask = cv2t.mask_from_comp( comp )
        cur_labels = mask*labels[t-1]
    #   labels_change(t, force_erode, cur_labels)

        gmask, n_cells = cv2t.add_divided_cells(comp, cur_labels, n_cells)
        rmask, n_cells = cv2t.add_new_cells(comp, cur_labels, n_cells)
        
        wsh = cv2t.cwatershed(img, cur_labels)
        labels[t] = cv2t.smooth_labels( wsh, force_erode ).astype('uint8')
        comp_out = CD.make_composite( labels[t], wsh, img, print_labels=True )
        comp_out *= mask[:,:,None]
        if rmask is not None:
            comp_out[rmask] = [255,0,0]
        if gmask is not None:
            comp_out[gmask] = [0,255,0]
        out[t] = comp_out
    else:
        out[t] = comp
else:
    iio.volwrite( imgname(comp_out_fn), out )

In [ ]:
gen = zip(range(T), source, labels)
wsh3d = np.empty(shape3d, dtype='uint16' )
for t, img, label in tqdm(gen, total=T):
    wsh3d[t] = cv2t.cwatershed(img, label)
else:
    iio.volwrite( imgname('-3wshed'), wsh3d )

In [ ]:
# Active Graph Network
import wflowtools.activegraph as AG; reload(AG);

def get_ag(j, w):
    blur = cv2.blur(j, (5,5))
    blur = ( blur-np.min(blur) )/( np.max(blur)-np.min(blur) )
    ag = AG.ActiveGraph( W=w, E=blur, fixed_edge_length = 50.0)
    n = 250
    d = 10
    prev = 15
    eps = 0.003
    alpha = 0.2
    for loop in range( int(n/d) ):
        for i in range(d):
            dr = ag.step( alpha = alpha )
        a = np.max( np.sum( dr**2, axis=1) )
        ag.redistribute()

#         if prev - a < eps * alpha:
#             break
#         else:
#             prev = a
    return ag

gs = []
# jxn08U = np.uint16( iio.volread( imgname('-0-8U') ) )
# wsh3d = np.uint8( iio.volread( imgname('-3wshed') ) )
for wsh, jxn in tqdm( zip( wsh3d, jxn08U ), total=T ):
    ag = get_ag( np.float64(jxn), wsh )
    ag.set_spacing(1.0)
    gs.append( ag )

gs = np.array(gs)
    
# import pickle
# with open( imgname('-agns', ext='pkl') , 'wb') as f:
#     pickle.dump( gs, f )


In [ ]:
import pickle
with open( imgname('-agns', ext='pkl') , 'wb') as f:
    pickle.dump( gs, f )

In [ ]:
t = 200
ax = plt.gca()
jxn    = jxn08U[t]
plt.imshow(  jxn, cmap = 'summer');
gs[t].plot(ax = ax, color='red')

In [ ]:
import wflowtools.celldrawing as CD; reload(CD);
fig = CD.init_figure(jxn08U[0], mult=1)
writer = iio.get_writer( imgname('_junctions', mid='', pfx=''))

try:
    for jxn, wsh, ag in tqdm( zip(jxn08U, wsh3d, gs) , total=gs.size ):
        ax = CD.get_new_axes(fig)
        ax.imshow(jxn)
        ag.plot(ax, color='red', linewidth=3)
        centers = CD.cell_centers( wsh )
        CD.draw_cells_labels(ax, wsh, centers=centers)
        data = CD.fig2data(fig)
        
        writer.append_data( data )
finally:
    writer.close()

In [ ]:
import pickle
import wflowtools.celldrawing as CD; reload(CD);

# wsh3d = iio.volread( imgname('-3wshed'))

centers_arr = []
for wsh in tqdm(wsh3d):
    centers_arr.append( CD.cell_centers(wsh) )
centers_arr = np.array( centers_arr )

with open( imgname('-wsh-centers', ext='pkl'), 'wb' ) as f:
    pickle.dump(centers_arr, f)

# del wsh3d

## B. Flares detection

In [ ]:
import scipy.ndimage as ndi
import scipy.interpolate as intpl
import wflowtools.activegraph as AG
reload(AG)

import pickle
with open( imgname('-agns', ext='pkl') , 'rb') as f:
    graphs = np.array(pickle.load( f ))
for ag in graphs:
    ag.__class__ = AG.ActiveGraph
with open( imgname('-agns', ext='pkl') , 'wb') as f:
    pickle.dump( graphs, f )
    
with open( imgname('-wsh-centers', ext='pkl'), 'rb' ) as f:
    centers_arr = pickle.load(f)

### Background + references

In [ ]:
fullimage = np.array( iio.volread(main_imgfile) )
rho_raw = fullimage[:,1,:,:]
del fullimage

import wflowtools.flared_epith as FE; reload(FE)

flepith = FE.FlaredEpithelium(rho_raw, graphs, tscale = TSCALE, lscale = LSCALE)

## Masking    
internal_mask3d = flepith.create_mask(
    circles_gen=flepith._internal_circles_gen(),
    lines_gen=flepith._all_edges_gen(),
)

In [ ]:
# # remove debris shadows
# T,H,W = rho_raw.shape
# img = np.ones_like(rho_raw[0], 'uint8')
# xx,yy = np.ogrid[0:H,0:W]
# img[(yy-216)**2+(xx-  7)**2 < 20**2] = 0
# img[(yy-298)**2+(xx-157)**2 < 15**2] = 0
# img[(yy-474)**2+(xx- 49)**2 < 15**2] = 0
# img[(yy-428)**2+(xx-152)**2 < 15**2] = 0
# img[(yy-297)**2+(xx-232)**2 < 11**2] = 0
# internal_mask3d *= img[None,:,:]

In [ ]:
#Findning underexpressing cells
wsh3d = np.array( iio.volread( imgname('-3wshed'))).astype('uint8')
cell_exp_levels = dict()
for i in range(1, wsh3d.max()+1):
    arr = rho_raw[wsh3d == i]
    if arr.size > 0:
        cell_exp_levels[i] = arr.mean()
plt.hist( list(cell_exp_levels.values()), bins=20, range=[0, 2500] );

In [ ]:
expression_mask3d = internal_mask3d.copy()
for i, expression_level in cell_exp_levels.items():
    if expression_level < 900:
        expression_mask3d[wsh3d == i] = 0
flepith.normalize(expression_mask3d);

In [ ]:
print( np.isnan( flepith.fdata3d ).sum(), flepith.fdata3d.min(), flepith.fdata3d.max())

In [ ]:
blur3d = ndi.gaussian_filter(flepith.fdata3d.astype('float'), (1,2,2)).astype('float32')

In [ ]:
from wflowtools import arr_util as AU; reload(AU);
iio.volwrite(imgname('-mask', pfx='a'), 255*expression_mask3d)
AU.write_logpack(main_imgfile.split('.tif')[0]+'_blur3d', blur3d)

In [ ]:
import wflowtools.flare as Flare; reload(Flare);
# for flare in flares:
#     flare.__class__ = Flare.Flare

In [ ]:
frame = 26
thr = 2.3
plt.imshow((blur3d[frame-1] > thr), cmap='gray')
graphs[frame-1].plot( plt.gca() )

In [ ]:
flares, labels3d = Flare.thresholded_flares(blur3d, thr)

In [ ]:
for flare in tqdm(flares):
    flare.data.category = None
    flare.calculate_intersections(graphs)
    flare.assign_category(graphs)
print( "%d good flares" % sum(1 for f in flares if f.data.type == 'good') )

In [ ]:
xlsx_filename = main_imgfile.split('.tif') [0] + '-flares.xlsx'
info = Flare.flares_info(flares)
info.to_excel( xlsx_filename )

In [ ]:
from collections import defaultdict
color_dict_red = defaultdict(lambda: ('none', 'brown', 'brown'))
color_dict_red['good'] = ('none', 'yellow', 'yellow')
import wflowtools.celldrawing as CD; reload(CD);

def fgen(flares, img3d, centers):
    fig = CD.init_figure(img3d[0], mult=1)
    shape2d = img3d.shape[1:]
    vmin, vmax = img3d.min(), img3d.max()
    for t, (img, cnts) in enumerate(zip(img3d, centers)):
        ax = CD.get_new_axes(fig)
        ax.imshow(img, cmap='gray', vmin=1.0, vmax=Flare.byte2float(255))
        for flare in flares:
            CD.draw_flare(ax, flare, t, shape=shape2d, color_dict=color_dict_red, labeling=True)
        CD.draw_cells_labels(ax, wsh=img, centers=cnts, color='gray', size=15)
        yield CD.fig2data(fig)

writer = (iio.get_writer( imgname('-flares_pre', pfx='a') ))
img_gen = fgen(flares, flepith.fdata3d.astype('float'), centers=centers_arr)
try:
    for data in tqdm(img_gen, total=graphs.size ):
        writer.append_data( data )
finally:
    writer.close()

In [ ]:
xlsx_filename = main_imgfile.split('.tif') [0] + '-flares.xlsx'
postflares = Flare.read_flares(xlsx_filename, blur3d)
print(len(postflares), ' flares')

In [ ]:
from collections import defaultdict
from itertools import combinations 

def filter_repeating_flares(flares, delta = 30):
    good_flares = defaultdict(list)
    for flare in flares:
        if flare.data.type == 'good':
            good_flares[flare.data.edge_label].append(flare)
    for arr in good_flares.values():
        for f1, f2 in combinations(arr, 2):
            if abs( int(f1.blob_rect[0,0]) - f2.blob_rect[0,0] ) < 30:
                print(f1.index, f2.index)
#                 f1.data.category = f1.data.category = '--repeating'
#                 f1.data.type = f1.data.type = 'repeating'

filter_repeating_flares(postflares)

In [ ]:
print( "%d good flares" % sum(1 for f in postflares if f.data.type == 'good') )

In [ ]:
writer =  iio.get_writer( imgname('_flares', pfx='', mid='') )
img_gen = fgen(postflares, flepith.fdata3d.astype('float'), centers=centers_arr)
try:
    for data in tqdm(img_gen, total=graphs.size ):
        writer.append_data( data )
finally:
    writer.close()

## C. Flare/edge geometry

In [ ]:
import pickle
with open( imgname('-agns', ext='pkl') , 'rb') as f:
    graphs = np.array(pickle.load( f ))
    
from wflowtools import arr_util as AU; reload(AU);
from glob import glob
from wflowtools import flare as Flare
fn = glob(main_imgfile.split('.tif')[0]+'_blur3d*')[0]
blur3d = AU.read_logpack(fn)

In [ ]:
xlsx_filename = main_imgfile.split('.tif') [0] + '-flares.xlsx'
flares = Flare.read_flares(xlsx_filename, blur3d)
print('%d / %d flares' % (sum(1 for f in flares if f.data.type == 'good'), len(flares)))
flares = [f for f in flares if f.data.type == 'good']
fdict = dict( (f.index, f) for f in flares  )

In [ ]:
### Setting directories

flares_dir = working_dir + '/_flares-data/'
if not os.path.isdir(flares_dir):
    os.makedirs(flares_dir)
    
for f in tqdm(flares):
    f.set_name( imgset_shortname + '-F%03d' % f.index )
    f.set_directory( flares_dir )
    f.add_edge_geometry(graphs, (TSCALE, LSCALE))

In [ ]:
import pickle
with open(imgname('-flares-dump', pfx='', mid='', ext='pkl'), 'wb') as f:
    pickle.dump(flares, f)

## D. First kymograms

In [ ]:
import pickle
with open( imgname('-agns', ext='pkl') , 'rb') as f:
    graphs = np.array(pickle.load( f ))
    
fullimage = np.array( iio.volread(main_imgfile) )
rho_raw = fullimage[:,1,:,:]
zo1_raw = fullimage[:,0,:,:]
act_raw = fullimage[:,2,:,:]
del fullimage

import wflowtools.flared_epith as FE;reload(FE);
flepith = FE.FlaredEpithelium(rho_raw, graphs, tscale = TSCALE, lscale = LSCALE)
mask3d = np.clip(np.array(iio.volread( imgname('-mask', pfx='a') )), a_min=0, a_max=1)
rp = FE.RhoProcessor(rho_raw, mask3d, flepith)
zp = FE.ZO1Processor(zo1_raw, mask3d, flepith, raw_kernel=FE.disk1x5x5)
ap = FE.ZO1Processor(act_raw, mask3d, flepith, raw_kernel=FE.disk1x5x5)
ap.ch = 'act'
# mask

In [ ]:
for flare in tqdm(flares):
    rp.add_kymo_signal(flare)
    zp.add_kymo_signal(flare)
    ap.add_kymo_signal(flare)
    FE.prepare_kymo(flare)

## E. Add channel singal

In [ ]:
def flare_step_D(flare):
    FE.fc_from_roi(flare)
    rp.add_flare_signal(flare)
    zp.add_flare_signal(flare)
    zp.add_total_signal(flare) 
    FE.align_fsdf_by_slope(flare.fsdf)
    FE.diff_jxn_len(flare.fsdf)

for flare in tqdm(good_flares):
    try:
        flare_step_D(flare)
    except SyntaxError:
        print(flare.index)

In [ ]:
import pickle
for flare in flares:
    with open(flare.directory + flare.name + '.pkl', 'wb') as f:
        pickle.dump(flare, f)

## E1. Add channel singal with 5px roi

In [ ]:
import pickle
with open( imgname('-agns', ext='pkl') , 'rb') as f:
    graphs = np.array(pickle.load( f ))
    
fullimage = np.array( iio.volread(main_imgfile) )
rho_raw = fullimage[:,0,:,:]
zo1_raw = fullimage[:,1,:,:]
del fullimage

import wflowtools.flared_epith as FE;reload(FE);
flepith = FE.FlaredEpithelium(rho_raw, graphs, tscale = TSCALE, lscale = LSCALE)
mask3d = np.clip(np.array(iio.volread( imgname('-mask', pfx='a') )), a_min=0, a_max=1)
rp = FE.RhoProcessor(rho_raw, mask3d, flepith, raw_kernel=FE.disk1x5x5)
zp = FE.ZO1Processor(zo1_raw, mask3d, flepith, raw_kernel=FE.disk1x5x5)
# mask

In [ ]:
import pickle
from glob import glob

flares_dir = working_dir + '/_flares-data/'
fns = glob(flares_dir + '*.pkl')
flares = []
for fn in fns:
    with open(fn, 'rb') as f:
        flares.append( pickle.load(f) )

In [ ]:
for flare in tqdm(flares):
    rp.add_flare_signal(flare)
    zp.add_flare_signal(flare)
    zp.add_total_signal(flare) 
    FE.align_fsdf_by_slope(flare.fsdf)
    FE.diff_jxn_len(flare.fsdf)


In [ ]:
flares_dir = working_dir + '/_flares-data_5px/'
if not os.path.isdir(flares_dir):
    os.makedirs(flares_dir)

In [ ]:
import pickle
for flare in flares:
    with open(flares_dir + flare.name + '.pkl', 'wb') as f:
        pickle.dump(flare, f)

## F. Aligning/pics

### Illustrations

In [ ]:
from wflowtools import celldrawing as CD;

# flares_dir = working_dir + '/_flares-data/'
folder = flares_dir + 'pics/'
if not os.path.isdir(folder):
    os.makedirs(folder)

In [ ]:
for flare in tqdm(flares):
    CD.edge_transformation(flare, flares_dir)

In [ ]:
def save_processed_kymo(flare):
    plt.cla()
 

    km = iio.volread( flare.data.kymo_filename )
    ch = km[0]
    arr = ch[ch > 0]
    vmin0 = np.percentile(arr, q=35)
    vmax0 = np.percentile(arr, q=99.9)

    ch = km[1]
    arr = ch[ch > 0]
    vmin1 = np.percentile(arr, q=5)
    vmax1 = np.percentile(arr, q=99.9)

    ch1 = np.uint8(255 * np.clip(
        (km[0] - vmin0) / (vmax0 - vmin0), a_min=0, a_max=1))
    ch2 = np.uint8(255 * np.clip(
        (km[1] - vmin1) / (vmax1 - vmin1), a_min=0, a_max=1))
    img = np.array([ch1, ch2, np.zeros_like(ch2)])
    dims, coords = FE.set_fc_path_from_roi(flare)

    t_list = flare.kmdf.index.get_level_values('t').unique()
    time = flare.fsdf.time_slope_align
    length = dims[0] * LSCALE
    t1,t2 = time[t_list[0]] , time[t_list[-1]]
    l1,l2 = -length/2, length/2,

    
    plt.imshow(img.transpose(1,2,0), extent=[l1,l2, t2,t1])
    
    coords = coords.transpose()-0.4
    plt.plot( coords[1]*LSCALE+l1, coords[0]/dims[1]*(t2-t1)+t1,'w')
    plt.gca().set_aspect(2*LSCALE/TSCALE)
    plt.ylabel('aligned time (s)')
    plt.xlabel('length ($\mu$m)')
    plt.ylim(t2,t1)
    plt.tight_layout()
    plt.savefig(folder + 'orgkm-%s.png' % flare.name,
               bbox_inches='tight')
for flare in tqdm(flares):
    save_processed_kymo(flare)
plt.clf();

In [ ]:
reload(CD)
img = rho_raw[0]
fig = CD.init_figure(img, mult=1)

def save_flare_screenshot(flare, flares):
    t = int( flare.fsdf.flare_s_min.dropna().index.values.mean()  )
    ax = CD.get_flare_img(fig, rho_raw[t], flare, flares, t, color_dict={'good': ('pink', 'red', 'white')})
    ax.text(0.01, 0.99,
                   ("t = %d" % t), fontsize=12,
                   transform=ax.transAxes,
                   verticalalignment='top', horizontalalignment='left')
    ax.set_axis_off()
    plt.savefig(folder + 'flare-shot-%s.png' % flare.name)
for flare in tqdm(flares):
    save_flare_screenshot(flare, [])
plt.clf();

### Plots and presentation

In [ ]:
import matplotlib
import pptx
from PIL import Image

cm = 360000

def get_defining_dimension(imgname, width, height):
    w, h = Image.open(imgname).size
    if( w/h > width/height ):
        return {'width': width}
    else:
        return {'height': height}

In [ ]:
def save_flare_plot(flare):
    plt.clf()
    ax = plt.gca()
    lns= []

    fsdf = flare.fsdf
    rho =  fsdf.rho_norm.dropna()
    index = rho.index
    time = fsdf.time_slope_align[index]
    ax.plot( time.iloc[[0,-1]], [1,1], '--', color='grey'  )

    ax.plot( time, rho, '.' , color='red', markersize=3)
    data = savgol_filter(rho.values, 11, 3)
    lns += ax.plot( time, data, '-', color='red', label="RhoA" );

    jxn = fsdf.jxn_norm[index]
    ax.plot( time, jxn, '.' , color='green', markersize=3)
    data = savgol_filter(jxn.values, 21, 3)
    lns += ax.plot( time, data, '-', color='green', label="ZO-1" );

    tot = fsdf.jxn_total[index] / fsdf.jxn_total[index].mean()
    ax.plot( time, tot, '.' , color='orange', markersize=3)
    data = savgol_filter(tot.values, 21, 3)
    lns += ax.plot( time, data, '-', color='orange', label="Total ZO-1" );

    ax.tick_params(axis='x', labelsize=15)
    ax.set_xlim(time.min(), time.max())
    ax.set_xlabel("aligned time (s)", fontsize=20)

    abstime = fsdf.time[index]
    axTop = ax.twiny()
    axTop.tick_params(axis='x', labelsize=12)
    axTop.set_xlim(abstime.min()/60, abstime.max()/60 )
    axTop.set_xlabel("absolute time (min)", fontsize=15)

    ax2 = ax.twinx()
    length = fsdf.jxn_length_diff_um[index]
    ax2.plot( time, length, '.', color='blue', label='_nolegend_', markersize=3)
    data = savgol_filter(length.values, 17, 3)
    lns += ax2.plot( time, data, '-', color='blue', label='Jxn length' )

    y1 = max(1 - min(rho.min(),jxn.min(),tot.min()), -length.min()/2)
    y2 = max(max(rho.max(),jxn.max(),tot.max()) - 1, length.max()/2)
    ax.set_ylim(1-y1, 1+y2)
    ax2.set_ylim(-2*y1, 2*y2)

    labs = [l.get_label() for l in lns]
    ax.legend(lns, labs, loc='upper left', fontsize=20)

    ax.set_ylabel("Normalized intensity", fontsize=20)
    ax.tick_params(axis='y', labelsize=15)

    ax2.set_ylabel("Change in length $\mu$m", fontsize=20)
    ax2.spines['right'].set_color('blue')
    ax2.yaxis.label.set_color('blue')
    ax2.tick_params(axis='y', colors='blue', labelsize=15)

    plt.savefig(folder + 'plot-%s.png' % flare.name,
               bbox_inches='tight')

for flare in tqdm(flares):
    save_flare_plot(flare)

In [ ]:
def fill_slide(slide, flare):
    tb = slide.shapes.add_textbox(
        left=4*cm, top=0.2*cm, width=3*cm, height=1*cm
    )
    tf = tb.text_frame
    tf.auto_size = True
    tf.text = "%s F%03d" % (imgset_shortname, flare.index)
    tf.paragraphs[0].font.name = 'Consolas'
    tf.paragraphs[0].font.size = 1*cm
    
    tb = slide.shapes.add_textbox(
        left=4*cm, top=0, width=3*cm, height=0.3*cm
    )
    tf = tb.text_frame
    tf.auto_size = True
    tf.text = "%s %s" % (imgset_type, imgset_filename)
    tf.paragraphs[0].font.name = 'Consolas'
    tf.paragraphs[0].font.size = 0.3*cm

    tb = slide.shapes.add_textbox(
        left=4*cm, top=1.3*cm, width=3*cm, height=0.7*cm
    )
    tf = tb.text_frame
    tf.auto_size = True
    if type(flare.data.edge_label) is tuple:
        tf.text = "jxn-%d-%d" % (flare.data.edge_label)
    else:
        tf.text = "/".join([("jxn-%d-%d" %  e) for e in flare.data.edge_label])
            
    tf.paragraphs[0].font.name = 'Consolas'
    tf.paragraphs[0].font.size = 0.5*cm

    plot = folder + 'plot-%s.png' % flare.name
    kymo = folder + 'orgkm-%s.png' % flare.name
    edge = folder + 'flareform-%s.png' % flare.name
    shot = folder + 'flare-shot-%s.png' % flare.name
    
    dim = get_defining_dimension( kymo, width=4*cm, height=9*cm )
    slide.shapes.add_picture(
        kymo,
        left =0,
        top = 0,
        **dim )
    
    dim = get_defining_dimension( edge, width=4*cm, height=6*cm )
    slide.shapes.add_picture(
        edge,
        left =4*cm,
        top = 3*cm,
        **dim )

    dim = get_defining_dimension( shot, width=4*cm, height=3*cm )
    slide.shapes.add_picture(
        shot,
        left =12*cm,
        top = 0,
        **dim )

    dim = get_defining_dimension( plot, width=8*cm, height=6*cm )
    slide.shapes.add_picture(
        plot,
        left =8*cm,
        top = 3*cm,
        **dim )

prs = pptx.Presentation()
prs.slide_height=9*cm
prs.slide_width=16*cm

slide_layout = prs.slide_layouts[6]
for flare in tqdm(flares):
    slide = prs.slides.add_slide(slide_layout)
    fill_slide(slide, flare)

prs.save(flares_dir + '/%s.pptx' % imgset_shortname)

# VTK

In [ ]:
import wflowtools.arr_util as AU
import scipy.ndimage as ndi
from evtk.hl import imageToVTK

with open("%s/%s-properties.txt" % (basedir, imgset_shortname) ) as f:
    img_properties = eval( f.read() )
spacing = (img_properties['y_delta'],img_properties['x_delta'],img_properties['t_delta'])

rho = iio.volread( imgname(pfx='a') )
rho = ndi.gaussian_filter(rho, (1, 2, 2))
rho = np.array( rho.transpose(2, 1, 0) )

rho = AU.logclip(rho)

scale = 20 #sec/um

imageToVTK("%s/%s-3d-blobs_%ds-um" % (basedir, imgset_shortname, scale),
        spacing = (spacing[0], spacing[1], spacing[2]/scale),
        pointData={'rho': rho})